The tutorial demonstrates how to create a new skill. It provides a step-by-step guide on creating a skill from scratch as well as using predefined templates

# Creating skill from existing template

If your goal is to create a skill that follows specific scenario (for example - classifying dataset rows onto predefined categories), then you can use one of the existing skills and redefine the instruction:

In [3]:
from adala.skills import ClassificationSkillWithCoT

new_skill = ClassificationSkillWithCoT(
    name='product_type',
    description='Skill to recognize general electronic product category',
    instructions='Recognize product category.',
    labels= [
        "Consumer Electronics",
        "Home Appliances",
        "Office Electronics",
        "Medical Electronics",
        "Industrial Electronics",
        "Automotive Electronics",
        "Wearable Electronics",
        "Networking and Communication",
        "Personal Care Electronics",
        "DIY and Hobbyist Electronics",
        "Home Automation and Security",
        "Portable Storage",
        "Power Devices"
    ]
)

Applying this skill to a dataset is straightforward:

In [13]:
from adala.agents import SingleShotAgent
from adala.datasets import DataFrameDataset

import pandas as pd

filepath = 'https://hs-sandbox-pub.s3.amazonaws.com/amazon_cells_labelled.tsv'
df = pd.read_csv(filepath, sep='\t', nrows=10)

agent = SingleShotAgent(
    dataset=DataFrameDataset(df=df),
    skill=new_skill
)

run_result = agent.run()
pd.concat((df, run_result.experience.predictions), axis=1)

100%|█████████████████████████████████████████████| 10/10 [00:01<00:00,  8.00it/s]


,text,label,ground_truth,predictions,score
0,So there is no way for me to plug it in here i...,0,NaN,Portable Storage,"{'Consumer Electronics': -0.84616333, 'Home Ap..."
1,"Good case, Excellent value.",1,NaN,Consumer Electronics,"{'Consumer Electronics': -0.9398417000000001, ..."
2,Great for the jawbone.,1,NaN,Wearable Electronics,"{'Consumer Electronics': -3.636905, 'Home Appl..."
3,Tied to charger for conversations lasting more...,0,NaN,Portable Storage,"{'Consumer Electronics': -7.305189, 'Home Appl..."
4,The mic is great.,1,NaN,Portable Storage,"{'Consumer Electronics': -1000, 'Home Applianc..."
5,I have to jiggle the plug to get it to line up...,0,NaN,Home Appliances,"{'Consumer Electronics': -1.1856575, 'Home App..."
6,If you have several dozen or several hundred c...,0,NaN,Personal Care Electronics,"{'Consumer Electronics': -5.035703, 'Home Appl..."
7,If you are Razr owner...you must have this!,1,NaN,Personal Care Electronics,"{'Consumer Electronics': -1.8599268999999998, ..."
8,"Needless to say, I wasted my money.",0,NaN,Consumer Electronics,"{'Consumer Electronics': -0.7968074, 'Home App..."
9,What a waste of money and time!.,0,NaN,Personal Care Electronics,"{'Consumer Electronics': -0.8627923000000001, ..."


# Creating a skill with new templates

You can modify skill by changing instructions, input and output templates. Let's explore the anatomy of the skill:

Input template is used to fetch data from the input dataset. It can additionally contain any other generations:

In [14]:
new_skill.input_template

"Input: {{text}}\nThoughts: {{gen 'rationale'}}\n"

Output template is where the model predictions are generated. To validate a specific prediction field against `ground_truth` data in the original dataset, specify `prediction_field` parameter in the skill:

In [16]:
new_skill.output_template

"Output: {{select 'predictions' options=labels logprobs='score'}}"

In [17]:
new_skill.prediction_field = 'predictions'

To read more how create templates, check [guidance documentation](https://github.com/guidance-ai/guidance)

# Create a custom skill


You can also create a completely new skill, by overriding the functions:

In [30]:
from adala.skills import Skill
from adala.datasets import Dataset
from adala.runtimes import Runtime
from adala.memories import ShortTermMemory, LongTermMemory

class MyNewSkill(Skill):
    
    def apply(self, dataset: Dataset, runtime: Runtime) -> ShortTermMemory:
        pass
    
    def evaluate(self, experience: ShortTermMemory) -> ShortTermMemory:
        pass
    
    def analyze(self, experience: ShortTermMemory, memory: LongTermMemory = None, runtime: Runtime = None) -> ShortTermMemory:
        pass
    
    def improve(self, experience: ShortTermMemory) -> ShortTermMemory:
        pass
    
MyNewSkill().model_dump()

{'name': '',
 'description': '',
 'instructions': '',
 'input_template': '',
 'output_template': '',
 'prediction_field': ''}